__The following code shows how the Synplots in Figure 2 were created__



5/9/19

I am attempting to redo my previous POLG analysis by following a more proper method for choosing my sequences. I am basing my approach off of Dinan et. al (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5644247/). For this study, I will obtain sequences by performing tblastn searches against the nr/nt NCBI database using selected reference species. 

--> I think it would be best to blast against the ref_seqRNA database rather than the nr/nt database

https://www.ncbi.nlm.nih.gov/homologene/2016 



My selected reference species' sequences will be

__Homo sapiens__: NM_002693.2, corresponding protein: NP_002684.1. TaxID Mammalia (taxid 40674).

__Gallus gallus__: XM_015292047.2 is the NCBI transcript for POLG but the Ensembl transcript, POLG-201 has the upstream CUG in the correct frame. I suspect that while this won't change my BLAST results, I'll use it for all the downstream analyses after BLAST and force the Ensembl version into my dictionary. Corresponding protein: XP_015147533.1. TaxID Sauropsid

__Xenopus tropicalis__: XM_002932235.4, corresponding protein: XP_002932281.2. TaxID Amphibia (taxid:8292)

__Danio rerio__: XM_001921095.6, corresponding protein: XP_001921130.3. TaxID teleost fishes (taxid:32443)

I am performing tblastn searches, using default parameters except I looked for the top 500 hits, with each of the corresponding protein sequences for each selected reference species using taxid Mammalia (taxid:40674), sauropsid, amphibian, teleost or vertebrate excluding mammalia, sauropsid, amphibian and teleost. Lastly the database is the RefSeqRNA database

Sequences that had 'partial mRNA' in the name were removed

__Mammalia investigation:__

These sequences were removed from synplot analysis due to the amount of gaps in their alignment from Mammalian

PREDICTED: Camelus bactrianus polymerase (DNA directed), gamma (POLG), partial mRNA
NCBI Reference Sequence: XM_010955340.1  --> This sequence is a partial mRNA sequence (the genomic assembly lacks the 5' portion of the CDS) and has thus been excluded in synplot and CUG kozak analysis. 

These sequences were also removed due to their gappyness and synplot wouldn't run: Bison_bison_bison(XM_010841133.1), Oryctolagus_cuniculus (XM_017337563), Camelus_ferus (XM_006192570)

__Sauropsid investigation__

Sequences that had 'partial mRNA' in the name were removed during the BLAST search. The synplot doesn't look terribly significant but Gallus_gallus has a stop codon free area in the +1 frame

__Amphibian investgiation__

Too few sequences present to do a proper synplot analysis. Xenopus_tropicalis doesn't have a stop codon free region in the +1 frame.

There are only three organisms that are analyzed

Xenopus_tropicalis:XM_002932235

Xenopus_laevis:XM_018250789

Nanorana_parkeri: XM_018571894

__Teleost fish investgiation__

Austrofundulus_limnaeus filtered to gappyness

The synplot doesn't look significant and there isn't a stop codon free area in the +1 frame for Danio_rerio




In [42]:
#BioPython is heavily used in this analysis with Python 3 (Anaconda distribution)

from Bio.Seq import Seq
from Bio import Entrez
from Bio import SeqIO
from Bio.Alphabet import IUPAC
import matplotlib
import matplotlib.pyplot as plt
% matplotlib inline
import numpy as np
from Bio.Align.Applications import MuscleCommandline
import os
from Bio import SeqIO
import csv
from itertools import islice
from Bio.Emboss.Applications import TranalignCommandline

__Before running the code below to generate tranaligned files, remember to change the __


1)BLAST_genbank, 

2)BLAST_textoutput, 

3)and which filter list is being used in the writeCDS and writeProtein commands. 


This portion of the code deals with generated tranaligned sequences__

In [43]:
cwd = os.getcwd()+os.sep

BLAST_genbank = 'Representative_Species/tblastn_refseqrna_mammalia_queryhomosapiens_1-12-20.gb'
BLAST_textoutput = 'Representative_Species/tblastn_refseqrna_mammalia_queryhomosapiens_1-12-20.txt'

muscle_executable = cwd+"muscle3.8.31_i86win32.exe"
output_all_CDS = cwd+"80_CDS_POLG_new.fasta"
output_protein = cwd+'80_POLG_protein.fasta'
output_all_CDS_ali = cwd+"80_POLG_muscle_out_new.clw"
ordered_CDS = cwd+'80_Ordered_CDS_POLG.fasta'
alignment_file = cwd+'80_POLG_muscle_alignment.fasta'

mammal_filter_list = ['Camelus_bactrianus','Bison_bison_bison','Oryctolagus_cuniculus','Camelus_ferus']
sauropsid_filter_list = []
teleost_filter_list = ['Austrofundulus_limnaeus']
amphibian_filter_list = []


tranalign_exe = r"C:\mEMBOSS\Tranalign.exe"
tranalignseq_out = cwd + 'POLG_tranalign_output_80.fasta'

In [44]:
#This is the POLG201 sequence for gallus gallus because the NCBI version has the CUG in the incorrect frame
custom_POLG201_Gallus_gallus_sequence = Seq('GTCGCCCCGGAGCCCCGCGTTGCACCGCGATCCGACCCGGGCGGCGCGGTGTGGGGGCGGGGGGGCGCGTGGGGACAGCGCGGGGCTGCACGGCGCGGGGAAGGGGAGGTGCGGAGCTTTGGGGGCGCGTGCAGAGCTGTGGGGAGCGGCGGGGCGTGCCGCGTGCCTTGCAGGGGTGCCGCGTGCTTTGCAGGGGTGCCGCGTGCGTGCAGAGGTGTTGCACGCCTTGCAGGGGTGCTGCGTGCTTCGACGCAGTGGTGCGCCCCAGCTGCGCATCCCGGCGCGCCGCACACCTACGAGGTGTCGCTTTTAGCAGCGCCGGTTGAAGCTCCCGCTGCGGACCCCTCCCCTCACCGCCGCTCTCCCCCTGCAGGAGGACCCCCCCCTTATCCGGCAGCACCGCGATGCTCCGCGCGCTCCGCCGAGGCTCAGCGCCGCGCCGCGCCGCCTCCCGGCCGTGCTCCGGGCCCTCCGCGCACCGCCCGCAGCCACGCGGCGACGAGGCCGAGCCGTCGGAGCGGAGCGAGCGCCGCGTGAACCCGCTGCACATCCAGATGTTGTCCCGGAACCTCCACGAGCAGATCTTCCGCGGGGCGCCCGTGCGGCACTCGGAGGCGGCCGTGCGGCGCAGCGTCGAACACCTGCAGCGGCACGGCCTGTGGGGCCGGCACGGCCCGTCGCTGCCCGACGTGAGTTTGCGCCTGCCCCGCATGTACGGCGCCGACATCGACGAGCATTTCCGCCGCCTGGCGCAGAAGCAGAGCCTGCCCTACCTGGAGGCGGCCGAGGAGCTGCTGCGCTGCCGCCTGCCCCCCGCACCACAGAGCTGGGCCCGGCAGCAGGGCTGGACGCGCTACGGCCCCGACGGGCGGCCCGAGGCGGTGGAGTGCCCGCGGGAGCGCGCGTTGGTGCTGGACGTGGAGGTGTGCGTGGCCGCCGGGCAGTGCCCCACTATGGCCGTGGCGGTGTCGCCGCACGCGTGGTACTCGTGGTGCAGCCGGCGCCTGCTGGAGCAGCGCTACTCGTGGGGCCCCCGGCTGGCGCTGCACGACCTCGTGCCTCTGGAGGGGACCGGCAGGCAGCAGGAGGGCGGCGAGAGGGTGGTGGTGGGGCACAACGTGGCTTTCGACCGCGCCTTCATCAGGGAGCAGTACCTCGTGCAGGGCTCCCGGGTGCGCTTCCTGGACACCATGAGCATGCACATGGCCATCTCGGGGCTCACGGGCTTCCAGCGCAGCCTCTGGATGGCCGCCAAGCACGGCAAGAGGAAGGGGCTGCAGCAGGTCAGGCAGCACATGAAGAAGACACGCAGCAAAGCCGAGGGGCCGGCGGTCTCTTCATGGGACTGGGTGCACGTCAGCAGCATCAACAACCTGGCAGATGTGCATGCACTGTACGTGGGAGGGGAACCGCTGCAGAAGGAGGCACGAGAGCTGTTTGTTAAGGGGACCATGGCTGACGTCAGGAATAACTTCCAGGAGCTGATGTCGTACTGTGCCAGCGATGTCCGGGCCACCTATGAGGTGTTCCAGGAGCAGCTGCCGCTCTTCATGGAGAGGTGCCCCCACCCCGTGACGTTTGCTGGGATGTTGGAGATGGGGGTGTCCTACCTGCCGGTCAACAGCAACTGGAGGAGGTACCTGGACGATGCTCAGGGCACCTATGAGGAGCTGCAGAAGGAGATGAAAAAGTCCTTGATGAACCTGGCCAACGATGCCTGCCAGCTGCTGCACGAGGACAGGTACAAGGAGGACCCCTGGCTCTGGGATCTGGAGTGGGACACGCAAGAGTTTAAGCAGAAGAAACCCGCTAAGAGGAAGAAGGATCAGAAAATAAACAGTGAAGCTTCCGAGACGGGCTCTGCTCAGGAGTGGAGGGAAGACCCCGGTCCCCCCAGCGAGGAGGAGGAGCTGAGAGCCCCCGAGAGCAGCACCTGCCTGGAGCGCCTGAAGGAGACGATCACACTGCAGCCCAAGAGGCTGCAGCACCTCCCGGGCCACCCGGGCTGGTACCGCAAGCTCTGCCCGCGCCTGGAGGAGGAGGGCTGGGTGCCGGGGCCCAGCCTCATCAGCCTGCAGATGCGGGTGACCCCGAAACTGATGCGCCTGGCCTGGGATGGCTTCCCTCTGCACTACTCGGAGAAGCACGGCTGGGGCTACCTGGTGCCGGGGCGGCAGGACAACCTGCCTGCAGCCTCTGCGGAGCCAGAGGGGCCTGTCTGCCCACACAGGGCGATCGAGCGGCTGTATCGGCAGCACTGCCTGCAGAGGGGCCAGGAGCAGCCCCCAGAGGAGGCTGGCGTGGAGGATGAGCTGATGGTGCTGGAGGGCAGCAGCATGTGGCAGAAGGTGGAGGAGCTGAGCCAGCTGGAGCTGGACATGGAGCGGCCGGGCAGGGCAGAGCAGAGCCAGATGCAGGATGAGGACGGGCTGCCAGAGCTGGTGGAGGAGAGCAGCCAGCCCTCATTCCACCACGGCAATGGCCCCTACAACGACGTCAACATCCCTGGATGCTGGTTCTTCAAGCTGCCCCACAAGGACGGCAATGAGAACAACGTGGGGAGCCCCTTTGCCAAGGACTTCCTGCCCCGCATGGAGGATGGCACGCTGCGGGCCACCGTGGGCCGCACCCATGGGACCAGAGCCCTGGAGATCAACAAGATGGTGTCCTTCTGGAGGAACGCTCACAAGCGGGTCAGTTCCCAGGTGGTTGTGTGGCTGAAGAAGGGGGAGCTGCCCCGTGCGGTGACCAGGCACCCGGCCTACAGCGAGGAGGAGGACTACGGGGCCATCCTGCCGCAGGTGGTGACTGCGGGTACCATCACCCGTCGGGCCGTGGAGCCCACGTGGCTGACAGCCAGCAATGCCCGGGCTGACCGTGTGGGCAGCGAGCTGAAGGCCATGGTCCAGGTGCCGCCCGGCTACTCTCTGGTGGGTGCAGATGTGGACTCCCAGGAGCTGTGGATAGCGGCGGTCCTGGGCGAGGCTCACTTTGCTGGCATGCACGGGTGCACGGCCTTCGGCTGGATGACCCTGCAAGGGAAGAAGAGCGACGGGACCGACCTGCATAGCAAGACGGCCGCCACGGTGGGCATCAGCCGGGAGCACGCCAAGGTCTTCAACTACGGGCGCATCTACGGGGCTGGGCAGCCCTTTGCCGAGCGGCTGCTGATGCAGTTCAATCACCGGCTGACACAGCAGCAGGCACGTGAGAAGGCACAGCAGATGTATGCAGTCACAAAGGGCATCCGGAGGTTTCATCTCAGCGAGGAGGGCGAGTGGCTGGTGAAGGAACTGGAGCTGGCTGTGGACAAAGCAGAAGATGGTACGGTGTCGGCCCAGGATGTGCAGAAGATCCAGAGAGAAGCCATGAGAAAGTCCCGAAGGAAGAAGAAGTGGGACGTGGTGGCTCACCGAATGTGGGCTGGAGGCACCGAGTCCGAAATGTTCAACAAGCTGGAGAGCATCGCTCTGTCCGCCTCGCCACAGACCCCGGTGCTGGGCTGTCATATCAGCAGGGCTCTGGAGCCTGCAGTGGCCAAAGGGGAGTTTCTAACCAGCAGAGTGAACTGGGTGGTGCAGAGCTCAGCTGTTGACTACCTGCACCTCATGCTGGTCTCCATGAAGTGGCTCTTTGAGGAGTATGACATAAATGGTCGCTTCTGCATCAGCATCCACGACGAGGTGCGCTACCTGGTGCAGGAGCAGGACCGCTACCGGGCAGCACTGGCCCTGCAGATCACCAACCTGCTCACACGGTGCATGTTTGCCTACAAGCTGGGCCTCCAGGATCTGCCGCAGTCCGTGGCTTTCTTCAGCGCTGTGGACATTGACCGGTGCTTAAGGAAGGAGGTGACCATGAACTGTGCGACTCCATCAAATCCAACCGGCATGGAGAAGAAGTACGGCATTCCTCGAGGAGAAGCACTGGATATATATCAGATAATTGAAATAACCAAAGGCTCACTGGAGAAGAAGTGATAACGTGAGAGTGCCAGAAGGTGCAAGTTGTCCAGAGAGCACACGGGAACCTGGCTGTCCTTTCAGAAGCACATACATGGCAGGGACCAATCCTGGTTGCGCCGCTTCCTTCTCGTGGTAAGAAAAAGATGTTCCTGATGAAGATTTTCATAGCAGCACATCTGAATGGGAGAGCTTGCATATTTGAATGGCTGGCAGCCAGCTTTAAGACCTGAGACACCTGACAGAGTCACTGCTTGCACACCCGTGGGGATGAAGAAAGAAGTCTTGAGTATTTGCCAGGAGACAGAATCAAATCAATCATCTGTACGTGCAGTTCTCCAAGACCAAGGTGAGGCTGCCACAGCACAGGTGCTGTAGGAGAAGGAGGTGGCAGCAGTTGCAAGCACACATTCTATTTTTTTCGCCTTCTTTTCTTTTGGGGTTCCTGGTTTTCATCTGGCTGCTCTGCTGTGCCGGACTGGAGAGAAATAGAGAGTTAAGAGTACCAAGTGTGAACGTTTGTGT')

In [45]:
#This function requires a genbank file from a blast result, the text_table of the results of a blast result. An optional
#parameter exists called optional_cutoff --> if an individual sorts sequences by something like query cover before downloading
#his/her blast result, the user can choose to stop processing at a specific sequence so that only sequences above a certain
#query cover are considered in downstream analysis. Alternatively, one could simply download sequences manually that are above
#a certain query cover
def processHitTable(genbank_file,text_table, optional_cutoff = ''):
    Sequence_dict = {}
    for file in SeqIO.parse(genbank_file, 'gb'):
        for feature in file.features:
                if feature.type == 'gene':
                    if 'gene' in feature.qualifiers.keys():
                        symbol = feature.qualifiers['gene']
                    if 'locus_tag' in feature.qualifiers.keys():
                        symbol = feature.qualifiers['locus_tag']
                if feature.type == 'source':
                    organism = feature.qualifiers['organism'][0].replace(" ", "_")
                    
                    #automatically should use POLG-201 transcript
                    if organism == 'Gallus_gallus':
                        Sequence_dict['Gallus_gallus'] = {}
                        Sequence_dict['Gallus_gallus']['POLG_201'] = {}
                        Sequence_dict['Gallus_gallus']['POLG_201']['nam'] = 'Ensembl_transcript_POLG-201'
                        Sequence_dict['Gallus_gallus']['POLG_201']['seq'] = (custom_POLG201_Gallus_gallus_sequence)
                        Sequence_dict['Gallus_gallus']['POLG_201']['start'] = 404
                        Sequence_dict['Gallus_gallus']['POLG_201']['end'] = 3983
                        Sequence_dict['Gallus_gallus']['POLG_201']['bit score'] = 1000000
                        
                if feature.type == 'CDS':
                    CDS = [int(a) for a in feature.location]
                    start = CDS[0]
                    end = CDS[-1]
                    accession = file.name
                    full_name = file.description
                    if organism not in Sequence_dict.keys():
                        Sequence_dict[organism] = dict()
                    Sequence_dict[organism][accession] = dict()
                    Sequence_dict[organism][accession]['nam'] = full_name
                    Sequence_dict[organism][accession]['seq'] = file.seq
                    Sequence_dict[organism][accession]['start'] = start
                    Sequence_dict[organism][accession]['end'] = end + 1
    
    final_hit_dict = {}
    with open(text_table) as f:
        reader = csv.DictReader(f, delimiter = "\t")
        for initial_row in islice(reader, 4, 5):
            header_list = str((initial_row['# tblastn'])).split('# Fields: ')[1].split(', ')
        hit_number = 0
        for row in islice(reader, 1, None):   
            hit_dict = {}
            hit_number +=1
            query_id = []
            query_id.append(str(row['# tblastn']))
            result_list = row[None]
            combined_results = query_id + result_list
            i = 0
            for item in header_list:
                hit_dict[item] = combined_results[i]
                i+=1  
            if optional_cutoff != '':
                if hit_dict['subject acc.ver'] == optional_cutoff:
                    break
            key = (hit_dict['subject acc.ver'].split('.'))[0]
            
            
            organism = ''
            accession = ''
            for item in Sequence_dict:
                for item2 in Sequence_dict[item]:
                    if item2 == key:
                        organism = item
                        accession = item2
                        Sequence_dict[organism][accession].update(hit_dict)
            
            
            
            
            #final_hit_dict[key] = hit_dict     
    return Sequence_dict

In [46]:
def bestHitPerOrganism(hitTable):
    singleHitDict = {}
    for organism in hitTable:
        bestScore = 0.0
        final_accession = ''
        transcript_variant = 0
        for accession in hitTable[organism]:
            current_score = float(hitTable[organism][accession]['bit score'])
            if current_score > bestScore:
                bestScore = current_score
                final_accession = accession
        singleHitDict[organism] = {'accession':final_accession, 'bit_score': bestScore,
                                   'sequence':hitTable[organism][final_accession]['seq'],
                                  'nam':hitTable[organism][final_accession]['nam'],
                                  'start':hitTable[organism][final_accession]['start'],
                                  'end':hitTable[organism][final_accession]['end']}
    return singleHitDict     

In [47]:
def writeCDS(out_file, D, skip = [],nam='organism'):
    text_file = open(out_file, 'w')
    for item in D:
        if item in skip:
            continue 
        if nam == 'organism':
            text_file.write(">%s\n%s\n" % (item,
                                           (D[item]['sequence'][D[item]['start']:D[item]['end']])))
        else:
            text_file.write(">%s\n%s\n" % (D[item][nam],
                                           D[item]['sequence'][D[item]['start']:D[item]['end']]))            
    text_file.close()

In [48]:
def writeProtein(out_file, D,skip = [], nam='organism'):
    text_file = open(out_file, 'w')
    for item in D:
        if item in skip:
            continue
        if nam == 'organism':
            text_file.write(">%s\n%s\n" % (item,
                                           ((D[item]['sequence'][D[item]['start']:D[item]['end']])).translate(to_stop=True)))
        else:
            text_file.write(">%s\n%s\n" % (D[item][nam],
                                           D[item]['sequence'][D[item]['start']:D[item]['end']]))            
    text_file.close()

In [49]:
def runMuscle(in_file, out_file, muscle_executable):
    muscle_cline = MuscleCommandline(muscle_executable, input=in_file, out=out_file)
    muscle_cline()

In [50]:
def readAlignment(in_file):
    alignment_dict = {}
    for seq_record in (SeqIO.parse(in_file, 'fasta')):
        name = seq_record.id
        seq = seq_record.seq
        alignment_dict[name] = str(seq)
    return (alignment_dict)

In [51]:
def orderCDSfile(final, alignment_dict, filename):
    CDS_order_file = open(filename,'w')
    for item in alignment_dict:
        CDS_order_file.write('>'+item+'\n'+str(final[item]['sequence'][final[item]['start']:final[item]['end']])+'\n')
    CDS_order_file.close()

In [52]:
def generateAlignmentFile(alignment_in, alignment_dict):
    alignment_file = open(alignment_in, 'w')
    for item in alignment_dict:
        alignment_file.write('>'+item+'\n'+(alignment_dict[item])+'\n')
    alignment_file.close()

In [53]:
def runTranalign(Tranalign_exe, orderedCDS_file_in, POLG_muscle_alignment_in, tranalignseq_out):
    needle_cline = TranalignCommandline(Tranalign_exe,asequence=orderedCDS_file_in,
                                 bsequence=POLG_muscle_alignment_in,
                                 stdout=True,outseq=tranalignseq_out)
    needle_cline()

In [54]:
hitTable = processHitTable(BLAST_genbank,BLAST_textoutput,80)
singleTranscriptTable = bestHitPerOrganism(hitTable)

In [55]:
writeCDS(output_all_CDS, singleTranscriptTable, mammal_filter_list)

In [56]:
writeProtein(output_protein, singleTranscriptTable, mammal_filter_list)

In [57]:
runMuscle(output_protein, output_all_CDS_ali, muscle_executable)

In [58]:
alignment_dict = readAlignment(output_all_CDS_ali)

In [59]:
orderCDSfile(singleTranscriptTable, alignment_dict,ordered_CDS)

In [60]:
generateAlignmentFile(alignment_file, alignment_dict)

In [61]:
runTranalign(tranalign_exe, ordered_CDS, alignment_file, tranalignseq_out)

In [62]:
tranaligned_dict = {}
for file in SeqIO.parse(tranalignseq_out, 'fasta'):
    tranaligned_dict[file.name] = str(file.seq)

In [63]:
#Run synplot on the online database: http://guinevere.otago.ac.nz/cgi-bin/aef/synplot.pl

In [41]:
#CleanupFiles
os.remove(output_all_CDS)
os.remove(output_protein)
os.remove(alignment_file)
os.remove(ordered_CDS)
os.remove(output_all_CDS_ali)
os.remove(tranalignseq_out)